In [ ]:
# If you intend to use this notebook, keep it in the same directory as manage.py

In [ ]:
import timeit
import re
import os
import django

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "rest.settings")
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

In [ ]:
import pandas as pd
from IPython.display import display

In [3]:
from django.db.models import Q, F, Count, Min, Max, Sum, Avg
from django.db.models.functions import Round
from django.db import connection
from customer_db.models import Customers, Orders, Products

In [4]:
def orm_to_df(record):
    try:
        iter(record)
    except:
        record = [record]
    if isinstance(record, dict):
        record = [record]
    df = pd.DataFrame.from_records(record)
    return df

In [5]:
def sql_to_df(raw_qs):
    if not isinstance(raw_qs, list):
        raw_qs = [raw_qs]
    df = pd.DataFrame(raw_qs)
    return df

In [6]:
from decimal import Decimal


def dec_to_float(record, precision=2):
    if isinstance(record, Decimal):
        return round(float(record), precision)
    return record

In [7]:
def round_(record, precision=2):
    if isinstance(record, float):
        return round(float(record), precision)
    return record

In [8]:
def equal(sqlq, ormq):
    try:
        iter(ormq)
    except:
        ormq = [ormq]

    if isinstance(ormq, dict):
        ormq = [ormq]

    ormq = list(ormq)

    if not isinstance(sqlq, list):
        sqlq = [sqlq]

    pattern = re.compile(r"(?<!^)(?=[A-Z])")
    sqlq = [
        {pattern.sub("_", k).lower(): round_(v) for k, v in dict_.items()}
        for dict_ in sqlq
    ]
    ormq = [{k: dec_to_float(v) for k, v in dict_.items()} for dict_ in ormq]
    if sqlq == ormq:
        print("Equal ✔️")
    else:
        print("Unequal ❌")

In [9]:
def dictfetchall(cursor):
    """
    Return all rows from a cursor as a dict.
    Assume the column names are unique.
    """
    columns = [col[0] for col in cursor.description]
    return [dict(zip(columns, row)) for row in cursor.fetchall()]

In [10]:
def sql_raw(raw_query):
    with connection.cursor() as c:
        sqlq = dictfetchall(c.execute(raw_query))
    return sqlq

In [11]:
sqlq = sql_raw("select * from Customers limit 3")
sql_to_df(sqlq)

,customerNumber,customerName,contactLastName,contactFirstName,phone,addressLine1,city,state,postalCode,country
0,103,Atelier graphique,Schmitt,Carine,40.32.2555,"54, rue Royale",Nantes,None,44000,France
1,112,Signal Gift Stores,King,Jean,7025551838,8489 Strong St.,Las Vegas,NV,83030,USA
2,114,"Australian Collectors, Co.",Ferguson,Peter,03 9520 4555,636 St Kilda Road,Melbourne,Victoria,3004,Australia


In [12]:
ormq = Customers.objects.all().values()[:3]
orm_to_df(ormq)

,customer_number,customer_name,contact_last_name,contact_first_name,phone,address_line1,city,state,postal_code,country
0,103,Atelier graphique,Schmitt,Carine,40.32.2555,"54, rue Royale",Nantes,None,44000,France
1,112,Signal Gift Stores,King,Jean,7025551838,8489 Strong St.,Las Vegas,NV,83030,USA
2,114,"Australian Collectors, Co.",Ferguson,Peter,03 9520 4555,636 St Kilda Road,Melbourne,Victoria,3004,Australia


In [13]:
equal(sqlq, ormq)

Equal ✔️


# W3SCHOOL SQL EXERCISES 
https://www.w3schools.com/sql/exercise.asp

Note: The equal function is guarateed to work, if you find an issue please open an new issue.

## SELECT
### Exercise 1
Insert the missing statement to get all the columns from the Customers table.

In [14]:
sqlq = sql_raw("select * from Customers")
ormq = Customers.objects.all().values()
equal(sqlq, ormq)

Equal ✔️


In [15]:
orm_to_df(ormq[:3])

,customer_number,customer_name,contact_last_name,contact_first_name,phone,address_line1,city,state,postal_code,country
0,103,Atelier graphique,Schmitt,Carine,40.32.2555,"54, rue Royale",Nantes,None,44000,France
1,112,Signal Gift Stores,King,Jean,7025551838,8489 Strong St.,Las Vegas,NV,83030,USA
2,114,"Australian Collectors, Co.",Ferguson,Peter,03 9520 4555,636 St Kilda Road,Melbourne,Victoria,3004,Australia


## Exercise 2
Write a statement that will select the City column from the Customers table.

In [16]:
sqlq = sql_raw("SELECT city FROM Customers;")
ormq = Customers.objects.all().values("city")
equal(sqlq, ormq)

Equal ✔️


In [17]:
orm_to_df(ormq[:3])

,city
0,Nantes
1,Las Vegas
2,Melbourne


### Exercise 3

In [18]:
sqlq = sql_raw("select distinct country from Customers;")
ormq = Customers.objects.all().values("country").distinct()
equal(sqlq, ormq)

Equal ✔️


In [19]:
orm_to_df(ormq[:3])

,country
0,France
1,USA
2,Australia


## WHERE
### Exercise 1
Select all records where the City column has the value "Berlin".

In [20]:
sqlq = sql_raw("select * from Customers where city='Berlin';")
ormq = Customers.objects.filter(city="Berlin").values()
equal(sqlq, ormq)

Equal ✔️


In [21]:
orm_to_df(ormq[:3])

,customer_number,customer_name,contact_last_name,contact_first_name,phone,address_line1,city,state,postal_code,country
0,307,Der Hund Imports,Andersen,Mel,030-0074555,Obere Str. 57,Berlin,None,12209,Germany


### Exercise 2

In [22]:
sqlq = sql_raw("select * from Customers where not city='Berlin';")
ormq = Customers.objects.exclude(city="Berlin").values()
equal(sqlq, ormq)

Equal ✔️


In [23]:
ormq1 = Customers.objects.filter(~Q(city="Berlin")).values()
equal(sqlq, ormq1)

Equal ✔️


In [24]:
orm_to_df(ormq[:3])

,customer_number,customer_name,contact_last_name,contact_first_name,phone,address_line1,city,state,postal_code,country
0,103,Atelier graphique,Schmitt,Carine,40.32.2555,"54, rue Royale",Nantes,None,44000,France
1,112,Signal Gift Stores,King,Jean,7025551838,8489 Strong St.,Las Vegas,NV,83030,USA
2,114,"Australian Collectors, Co.",Ferguson,Peter,03 9520 4555,636 St Kilda Road,Melbourne,Victoria,3004,Australia


### Exercise 3
Select all records where the Customer_number column has the value 320.

In [25]:
sqlq = sql_raw("select * from Customers where customerNumber='320';")
ormq = Customers.objects.filter(customer_number=320).values()
equal(sqlq, ormq)

Equal ✔️


In [26]:
orm_to_df(ormq[:3])

,customer_number,customer_name,contact_last_name,contact_first_name,phone,address_line1,city,state,postal_code,country
0,320,Mini Creations Ltd.,Huang,Wing,5085559555,4575 Hillside Dr.,New Bedford,MA,50553,USA


### Exercise 4
Select all records where the City column has the value 'Berlin' and the PostalCode column has the value 12209.

In [27]:
sqlq = sql_raw("SELECT * FROM customers WHERE city='Berlin' and postalCode=12209;")
ormq = Customers.objects.filter(city="Berlin", postal_code=12209).values()
equal(sqlq, ormq)

Equal ✔️


In [28]:
orm_to_df(ormq)

,customer_number,customer_name,contact_last_name,contact_first_name,phone,address_line1,city,state,postal_code,country
0,307,Der Hund Imports,Andersen,Mel,030-0074555,Obere Str. 57,Berlin,None,12209,Germany


### Exercise 5
Select all records where the City column has the value 'Berlin' or 'London'. 

In [29]:
sqlq = sql_raw("select * from Customers where city='Berlin' or city='London'")
ormq = Customers.objects.filter(Q(city="Berlin") | Q(city="London")).values()
equal(sqlq, ormq)

Equal ✔️


In [30]:
orm_to_df(ormq)

,customer_number,customer_name,contact_last_name,contact_first_name,phone,address_line1,city,state,postal_code,country
0,307,Der Hund Imports,Andersen,Mel,030-0074555,Obere Str. 57,Berlin,None,12209,Germany
1,324,"Stylish Desk Decors, Co.",Brown,Ann,(171) 555-0297,35 King George,London,None,WX3 6FW,UK
2,489,"Double Decker Gift Stores, Ltd",Smith,Thomas,(171) 555-7555,120 Hanover Sq.,London,None,WA1 1DP,UK


## ORDER BY
### Exercise 1
Select all records from the Customers table, sort the result alphabetically by the column City.

In [31]:
sqlq = sql_raw("SELECT * FROM Customers ORDER BY city")
ormq = Customers.objects.all().order_by("city").values()
equal(sqlq, ormq)

Equal ✔️


In [32]:
orm_to_df(ormq[:3])

,customer_number,customer_name,contact_last_name,contact_first_name,phone,address_line1,city,state,postal_code,country
0,459,Warburg Exchange,Ottlieb,Sven,0241-039123,Walserweg 21,Aachen,None,52066,Germany
1,157,Diecast Classics Inc.,Leong,Kelvin,2155551555,7586 Pompton St.,Allentown,PA,70267,USA
2,303,Schuyler Imports,Schuyler,Bradley,+31 20 491 9555,Kingsfordweg 151,Amsterdam,None,1043 GR,Netherlands


### Exercise 2
Select all records from the Customers table, sort the result reversed alphabetically by the column City.

In [33]:
sqlq = sql_raw("SELECT * FROM Customers ORDER BY city desc")
ormq = Customers.objects.all().order_by("-city").values()
equal(sqlq, ormq)

Equal ✔️


In [34]:
orm_to_df(ormq[:3])

,customer_number,customer_name,contact_last_name,contact_first_name,phone,address_line1,city,state,postal_code,country
0,227,Heintze Collectables,Ibsen,Palle,86 21 3555,Smagsloget 45,┼rhus,None,8200,Denmark
1,319,Mini Classics,Frick,Steve,9145554562,3758 North Pendale Street,White Plains,NY,24067,USA
2,412,"Extreme Desk Decorations, Ltd",McRoy,Sarah,04 499 9555,101 Lambton Quay,Wellington,None,None,New Zealand


### Exercise 3
Select all records from the Customers table, sort the result alphabetically,
first by the column Country, then, by the column City.

In [35]:
sqlq = sql_raw("SELECT * FROM Customers ORDER BY country, city")
ormq = Customers.objects.all().order_by("country", "city").values()
equal(sqlq, ormq)

Equal ✔️


In [36]:
orm_to_df(ormq[:3])

,customer_number,customer_name,contact_last_name,contact_first_name,phone,address_line1,city,state,postal_code,country
0,282,Souveniers And Things Co.,Huxley,Adrian,+61 2 9495 8555,Monitor Money Building,Chatswood,NSW,2067,Australia
1,471,"Australian Collectables, Ltd",Clenahan,Sean,61-9-3844-6555,7 Allen Street,Glen Waverly,Victoria,3150,Australia
2,114,"Australian Collectors, Co.",Ferguson,Peter,03 9520 4555,636 St Kilda Road,Melbourne,Victoria,3004,Australia


## NULL
### Exercise 1
Select all records from the Customers where the PostalCode column is empty.

In [37]:
sqlq = sql_raw("SELECT * FROM Customers WHERE postalCode IS null")
ormq = Customers.objects.filter(postal_code=None).values()
equal(sqlq, ormq)

Equal ✔️


In [38]:
orm_to_df(ormq[:3])

,customer_number,customer_name,contact_last_name,contact_first_name,phone,address_line1,city,state,postal_code,country
0,211,"King Kong Collectables, Co.",Gao,Mike,+852 2251 1555,Bank of China Tower,Central Hong Kong,None,None,Hong Kong
1,323,"Down Under Souveniers, Inc",Graham,Mike,+64 9 312 5555,162-164 Grafton Road,Auckland,None,None,New Zealand
2,348,"Asian Treasures, Inc.",McKenna,Patricia,2967 555,8 Johnstown Road,Cork,Co. Cork,None,Ireland


### Exercise 2
Select all records from the Customers where the PostalCode column is NOT empty.

In [39]:
sqlq = sql_raw("select * from Customers where postalCode is not null")
ormq = Customers.objects.exclude(postal_code=None).values()
equal(sqlq, ormq)

Equal ✔️


In [40]:
ormq1 = Customers.objects.filter(~Q(postal_code=None)).values()
equal(sqlq, ormq1)

Equal ✔️


In [41]:
orm_to_df(ormq[:3])

,customer_number,customer_name,contact_last_name,contact_first_name,phone,address_line1,city,state,postal_code,country
0,103,Atelier graphique,Schmitt,Carine,40.32.2555,"54, rue Royale",Nantes,None,44000,France
1,112,Signal Gift Stores,King,Jean,7025551838,8489 Strong St.,Las Vegas,NV,83030,USA
2,114,"Australian Collectors, Co.",Ferguson,Peter,03 9520 4555,636 St Kilda Road,Melbourne,Victoria,3004,Australia


## FUNCTION
### Exercise 1
Use the MIN function to select the record with the smallest value of the Price column.

In [42]:
sqlq = sql_raw("select MIN(buyPrice) mn from products")
ormq = Products.objects.aggregate(mn=Min("buy_price"))
equal(sqlq, ormq)

Equal ✔️


In [43]:
orm_to_df(ormq)

,mn
0,15.9100000000000


### Exercise 2
Use an SQL function to select the record with the highest value of the Price column.

In [44]:
sqlq = sql_raw("select MAX(buyPrice) mn from products")
ormq = dec_to_float(Products.objects.aggregate(mn=Max("buy_price")))
equal(sqlq, ormq)

Equal ✔️


In [45]:
orm_to_df(ormq)

,mn
0,103.420000000000


### Excercise 3
Use the correct function to return the number of records that have the Price value set to 15.91.

In [46]:
sqlq = sql_raw("select COUNT(*) cnt from products WHERE buyPrice=15.91")
ormq = (
    Products.objects.filter(buy_price=15.91)
    .annotate(cnt=Count("product_code"))
    .values("cnt")
)
equal(sqlq, ormq)

Equal ✔️


In [47]:
ormq1 = Products.objects.filter(buy_price=15.91).count()
ormq1 = [{"cnt": ormq1}]
equal(sqlq, ormq1)

Equal ✔️


In [48]:
orm_to_df(ormq)

,cnt
0,1


### Exercise 4
Use an SQL function to calculate the average price of all products.

In [49]:
sqlq = dec_to_float(sql_raw("SELECT AVG(buyPrice) mean FROM products;"))
ormq = dec_to_float(Products.objects.aggregate(mean=Avg("buy_price")))
equal(sqlq, ormq)

Equal ✔️


In [50]:
orm_to_df(ormq)

,mean
0,54.3951818181818


### Exercise 5
Use an SQL function to calculate the sum of all the Price column values in the Products table.

In [51]:
sqlq = dec_to_float(sql_raw("SELECT SUM(buyPrice) sum_ FROM products;"))
ormq = dec_to_float(Products.objects.aggregate(sum_=Sum("buy_price")))
equal(sqlq, ormq)

Equal ✔️


In [52]:
orm_to_df(ormq)

,sum_
0,5983.47000000000


## LIKE
### Exercise 1
Select all records where the value of the City column starts with the letter "a".

In [53]:
sqlq = sql_raw("SELECT * FROM customers WHERE city LIKE 'a%'")
ormq = Customers.objects.filter(city__istartswith="a").values()
equal(sqlq, ormq)

Equal ✔️


In [54]:
orm_to_df(ormq[:3])

,customer_number,customer_name,contact_last_name,contact_first_name,phone,address_line1,city,state,postal_code,country
0,157,Diecast Classics Inc.,Leong,Kelvin,2155551555,7586 Pompton St.,Allentown,PA,70267,USA
1,303,Schuyler Imports,Schuyler,Bradley,+31 20 491 9555,Kingsfordweg 151,Amsterdam,None,1043 GR,Netherlands
2,323,"Down Under Souveniers, Inc",Graham,Mike,+64 9 312 5555,162-164 Grafton Road,Auckland,None,None,New Zealand


### Exercise 2
Select all records where the value of the City column ends with the letter "a".

In [55]:
sqlq = sql_raw("SELECT * FROM customers WHERE city LIKE '%a'")
ormq = Customers.objects.filter(city__iendswith="a").values()
equal(sqlq, ormq)

Equal ✔️


In [56]:
orm_to_df(ormq[:3])

,customer_number,customer_name,contact_last_name,contact_first_name,phone,address_line1,city,state,postal_code,country
0,125,Havel & Zbyszek Co,Piestrzeniewicz,Zbyszek,(26) 642-7555,ul. Filtrowa 68,Warszawa,None,01-012,Poland
1,169,Porto Imports Co.,de Castro,Isabel,(1) 356-5555,Estrada da sa·de n. 58,Lisboa,None,1756,Portugal
2,205,Toys4GrownUps.com,Young,Julie,6265557265,78934 Hillside Dr.,Pasadena,CA,90003,USA


### Exercise 3
Select all records where the value of the City column contains the letter "a".

In [57]:
sqlq = sql_raw("SELECT * FROM customers WHERE city LIKE '%a%'")
ormq = Customers.objects.filter(city__icontains="a").values()
equal(sqlq, ormq)

Equal ✔️


In [58]:
orm_to_df(ormq[:3])

,customer_number,customer_name,contact_last_name,contact_first_name,phone,address_line1,city,state,postal_code,country
0,103,Atelier graphique,Schmitt,Carine,40.32.2555,"54, rue Royale",Nantes,None,44000,France
1,112,Signal Gift Stores,King,Jean,7025551838,8489 Strong St.,Las Vegas,NV,83030,USA
2,119,La Rochelle Gifts,Labrune,Janine,40.67.8555,"67, rue des Cinquante Otages",Nantes,None,44000,France


### Exercise 4
Select all records where the value of the City column starts with letter "a" and ends with the letter "b".
(Note no such rows exists, so we will use L%n

In [59]:
sqlq = sql_raw("SELECT * FROM customers WHERE city LIKE 'L%n'")
ormq = Customers.objects.filter(
    Q(city__istartswith="L") & Q(city__endswith="n")
).values()
equal(sqlq, ormq)

Equal ✔️


In [60]:
ormq1 = Customers.objects.filter(city__regex=r"^L.*n$").values()
equal(sqlq, ormq1)

Equal ✔️


In [61]:
orm_to_df(ormq[:3])

,customer_number,customer_name,contact_last_name,contact_first_name,phone,address_line1,city,state,postal_code,country
0,146,"Saveley & Henriot, Co.",Saveley,Mary,78.32.5555,"2, rue du Commerce",Lyon,None,69004,France
1,324,"Stylish Desk Decors, Co.",Brown,Ann,(171) 555-0297,35 King George,London,None,WX3 6FW,UK
2,489,"Double Decker Gift Stores, Ltd",Smith,Thomas,(171) 555-7555,120 Hanover Sq.,London,None,WA1 1DP,UK


### Exercise 5
Select all records where the value of the City column does NOT start with the letter "a".

In [62]:
sqlq = sql_raw("SELECT * FROM customers WHERE city NOT LIKE 'a%'")
ormq = Customers.objects.exclude(city__istartswith="a").values()
equal(sqlq, ormq)

Equal ✔️


In [63]:
ormq1 = Customers.objects.filter(city__regex=r"^[^aA]").values()
equal(sqlq, ormq1)

Equal ✔️


In [64]:
ormq2 = Customers.objects.filter(~Q(city__istartswith="a")).values()
equal(sqlq, ormq2)

Equal ✔️


In [65]:
orm_to_df(ormq[:3])

,customer_number,customer_name,contact_last_name,contact_first_name,phone,address_line1,city,state,postal_code,country
0,103,Atelier graphique,Schmitt,Carine,40.32.2555,"54, rue Royale",Nantes,None,44000,France
1,112,Signal Gift Stores,King,Jean,7025551838,8489 Strong St.,Las Vegas,NV,83030,USA
2,114,"Australian Collectors, Co.",Ferguson,Peter,03 9520 4555,636 St Kilda Road,Melbourne,Victoria,3004,Australia


## Wilcards
### Exercise 1
Select all records where the second letter of the City is an "a".

In [66]:
sqlq = sql_raw("SELECT * FROM customers WHERE city LIKE '_a%'")
ormq = Customers.objects.filter(city__regex=r"^.[aA].*$").values()
equal(sqlq, ormq)

Equal ✔️


In [67]:
orm_to_df(ormq[:3])

,customer_number,customer_name,contact_last_name,contact_first_name,phone,address_line1,city,state,postal_code,country
0,103,Atelier graphique,Schmitt,Carine,40.32.2555,"54, rue Royale",Nantes,None,44000,France
1,112,Signal Gift Stores,King,Jean,7025551838,8489 Strong St.,Las Vegas,NV,83030,USA
2,119,La Rochelle Gifts,Labrune,Janine,40.67.8555,"67, rue des Cinquante Otages",Nantes,None,44000,France


### Exercise 2
Select all records where the first letter of the City is an "a" or a "c" or an "s".

In [68]:
# sqlq =  Customers.objects.raw("SELECT * FROM customers WHERE city LIKE '[acs]%'")
sqlq = sql_raw(
    "SELECT * FROM customers WHERE CITY LIKE 'a%' OR CITY LIKE 'c%' OR CITY LIKE 's%'"
)
query = Q()
for ch in ["a", "c", "s"]:
    query = query | Q(city__istartswith=ch)
ormq = Customers.objects.filter(query).values()
equal(sqlq, ormq)

Equal ✔️


In [69]:
ormq1 = Customers.objects.filter(city__regex=r"^[acsACS].*$").values()
equal(sqlq, ormq1)

Equal ✔️


In [70]:
orm_to_df(ormq[:3])

,customer_number,customer_name,contact_last_name,contact_first_name,phone,address_line1,city,state,postal_code,country
0,121,Baane Mini Imports,Bergulfsen,Jonas,07-98 9555,Erling Skakkes gate 78,Stavern,None,4110,Norway
1,124,Mini Gifts Distributors Ltd.,Nelson,Susan,4155551450,5677 Strong St.,San Rafael,CA,97562,USA
2,129,Mini Wheels Co.,Murphy,Julie,6505555787,5557 North Pendale Street,San Francisco,CA,94217,USA


### Exercise 3
Select all records where the first letter of the City starts with anything from an "a" to an "f".

In [71]:
# sqlq =  Customers.objects.raw("SELECT * FROM customers WHERE city LIKE '[a-f]%'") # doesnt work in sqlite
sqlq = sql_raw("SELECT * FROM customers WHERE city REGEXP '^[a-fA-F].*$'")
ormq = Customers.objects.filter(city__regex=r"^[a-fA-F].*$").values()
equal(sqlq, ormq)

Equal ✔️


### Exercise 4
Select all records where the first letter of the City is NOT an "a" or a "c" or an "f".

In [72]:
# sqlq =  Customers.objects.raw("SELECT * FROM customers WHERE city LIKE '[!acf]%'")
sqlq = sql_raw(
    "SELECT * FROM customers WHERE CITY NOT LIKE 'a%' OR CITY NOT LIKE 'c%' OR CITY NOT LIKE 's%'"
)
query = Q()
for ch in ["a", "c", "d"]:
    query = query | ~Q(city__istartswith=ch)
ormq = Customers.objects.filter(query).values()
equal(sqlq, ormq)

Equal ✔️


In [73]:
orm_to_df(ormq[:3])

,customer_number,customer_name,contact_last_name,contact_first_name,phone,address_line1,city,state,postal_code,country
0,103,Atelier graphique,Schmitt,Carine,40.32.2555,"54, rue Royale",Nantes,None,44000,France
1,112,Signal Gift Stores,King,Jean,7025551838,8489 Strong St.,Las Vegas,NV,83030,USA
2,114,"Australian Collectors, Co.",Ferguson,Peter,03 9520 4555,636 St Kilda Road,Melbourne,Victoria,3004,Australia


# IN
### Exercise 1
Use the IN operator to select all the records where Country is either "Norway" or "France".

In [74]:
sqlq = sql_raw("SELECT * FROM customers WHERE country IN ('Norway', 'France')")
ormq = Customers.objects.filter(country__in=["Norway", "France"]).values()
equal(sqlq, ormq)

Equal ✔️


In [75]:
orm_to_df(ormq[:3])

,customer_number,customer_name,contact_last_name,contact_first_name,phone,address_line1,city,state,postal_code,country
0,103,Atelier graphique,Schmitt,Carine,40.32.2555,"54, rue Royale",Nantes,None,44000,France
1,119,La Rochelle Gifts,Labrune,Janine,40.67.8555,"67, rue des Cinquante Otages",Nantes,None,44000,France
2,121,Baane Mini Imports,Bergulfsen,Jonas,07-98 9555,Erling Skakkes gate 78,Stavern,None,4110,Norway


### Exercise 2
Use the IN operator to select all the records where Country is NOT "Norway" and NOT "France".

In [76]:
sqlq = sql_raw("SELECT * FROM customers WHERE country NOT IN ('Norway', 'France')")
ormq = Customers.objects.exclude(country__in=["Norway", "France"]).values()
equal(sqlq, ormq)

Equal ✔️


In [77]:
ormq1 = Customers.objects.filter(~Q(country__in=["Norway", "France"])).values()
equal(sqlq, ormq1)

Equal ✔️


In [78]:
orm_to_df(ormq[:3])

,customer_number,customer_name,contact_last_name,contact_first_name,phone,address_line1,city,state,postal_code,country
0,112,Signal Gift Stores,King,Jean,7025551838,8489 Strong St.,Las Vegas,NV,83030,USA
1,114,"Australian Collectors, Co.",Ferguson,Peter,03 9520 4555,636 St Kilda Road,Melbourne,Victoria,3004,Australia
2,124,Mini Gifts Distributors Ltd.,Nelson,Susan,4155551450,5677 Strong St.,San Rafael,CA,97562,USA


# BETWEEN
### Exercise 1
Use the BETWEEN operator to select all the records where the value of the Price column is between 10 and 20.

In [79]:
sqlq = sql_raw("SELECT * FROM products WHERE buyPrice BETWEEN 10 AND 20;")
ormq = Products.objects.filter(Q(buy_price__gte=10) & Q(buy_price__lte=20)).values()
equal(sqlq, ormq)

Equal ✔️


In [80]:
orm_to_df(ormq[:3])

,product_code,product_name,product_scale,product_vendor,product_description,quantity_in_stock,buy_price,msrp
0,S24_2840,1958 Chevy Corvette Limited Edition,1:24,Carousel DieCast Legends,The operating parts of this 1958 Chevy Corvett...,2542,15.91,35.36
1,S24_2972,1982 Lamborghini Diablo,1:24,Second Gear Diecast,"This replica features opening doors, superb de...",7723,16.24,37.76


### Exercise 2
Use the BETWEEN operator to select all the records where the value of the ProductName column is alphabetically between 'Geitost' and 'Pavlova'.

In [81]:
sqlq = sql_raw("SELECT * FROM products WHERE buyPrice NOT BETWEEN 10 AND 20;")
ormq = Products.objects.filter(~Q(buy_price__gte=10) | ~Q(buy_price__lte=20)).values()
equal(sqlq, ormq)

Equal ✔️


In [82]:
orm_to_df(ormq[:3])

,product_code,product_name,product_scale,product_vendor,product_description,quantity_in_stock,buy_price,msrp
0,S10_1678,1969 Harley Davidson Ultimate Chopper,1:10,Min Lin Diecast,"This replica features working kickstand, front...",7933,48.81,95.70
1,S10_1949,1952 Alpine Renault 1300,1:10,Classic Metal Creations,Turnable front wheels; steering function; deta...,7305,98.58,214.30
2,S10_2016,1996 Moto Guzzi 1100i,1:10,Highway 66 Mini Classics,"Official Moto Guzzi logos and insignias, saddl...",6625,68.99,118.94


### Exercise 3
Use the BETWEEN operator to select all the records
where the value of the ProductName column is alphabetically between '1904 Buick Runabout' and '1928 Mercedes-Benz SSK'.

In [83]:
sqlq = sql_raw(
    "SELECT * FROM products WHERE productName BETWEEN '1904 Buick Runabout' AND '1928 Mercedes-Benz SSK';"
)
ormq = Products.objects.filter(
    Q(product_name__gte="1904 Buick Runabout")
    & Q(product_name__lte="1928 Mercedes-Benz SSK")
).values()
equal(sqlq, ormq)

Equal ✔️


In [84]:
orm_to_df(ormq[:3])

,product_code,product_name,product_scale,product_vendor,product_description,quantity_in_stock,buy_price,msrp
0,S18_1749,1917 Grand Touring Sedan,1:18,Welly Diecast Productions,This 1:18 scale replica of the 1917 Grand Tour...,2724,86.70,170.00
1,S18_2248,1911 Ford Town Car,1:18,Motor City Art Classics,"Features opening hood, opening doors, opening ...",540,33.30,60.54
2,S18_2432,1926 Ford Fire Engine,1:18,Carousel DieCast Legends,Gleaming red handsome appearance. Everything i...,2018,24.92,60.77


# ALIAS
### Exercise 1
When displaying the Customers table, make an ALIAS of the PostalCode column, the column should be called Pno instead.

In [85]:
sqlq = sql_raw("SELECT customerName, addressLine1, postalCode pno FROM customers;")
ormq = Customers.objects.values("customer_name", "address_line1", pno=F("postal_code"))
equal(sqlq, ormq)

Equal ✔️


In [86]:
orm_to_df(ormq[:3])

,customer_name,address_line1,pno
0,Atelier graphique,"54, rue Royale",44000
1,Signal Gift Stores,8489 Strong St.,83030
2,"Australian Collectors, Co.",636 St Kilda Road,3004


### Exercise 2
When displaying the Customers table, refer to the table as Consumers instead of Customers.

In [87]:
# Couldn't find an answer

# JOIN
### Exercise 1
Insert the missing parts in the JOIN clause to join the two tables Orders and Customers,
using the CustomerID field in both tables as the relationship between the two tables.

In [88]:
q = """
Select * FROM orders o
LEFT OUTER JOIN customers c
on o.customerNumber == c.customerNumber
"""
sqlq = sql_raw(q)
ji = sql_to_df(sqlq)
print(ji.shape)
ji.head(2)

(326, 16)


,orderNumber,orderDate,requiredDate,shippedDate,status,comments,customerNumber,customerName,contactLastName,contactFirstName,phone,addressLine1,city,state,postalCode,country
0,10100,2003-01-06,2003-01-13,2003-01-10,Shipped,None,363,Online Diecast Creations Co.,Young,Dorothy,6035558647,2304 Long Airport Avenue,Nashua,NH,62005,USA
1,10101,2003-01-09,2003-01-18,2003-01-11,Shipped,Check on availability.,128,"Blauer See Auto, Co.",Keitel,Roland,+49 69 66 90 2555,Lyonerstr. 34,Frankfurt,None,60528,Germany


In [89]:
def get_all_join_fields(lh_class, rh_class, fk):
    lh_class_fields = [field.name for field in lh_class._meta.local_fields]
    rh_class_fields = {
        field.name: F(f"{fk}__{field.name}")
        for field in rh_class._meta.local_fields
        if field.name not in lh_class_fields
    }
    return lh_class_fields, rh_class_fields

In [90]:
lh_fields, rh_field_dict = get_all_join_fields(Orders, Customers, "customer_number")
# this will perform an inner join because customer_number is non nullable
ormq = Orders.objects.select_related("customer_number").values(
    *lh_fields, **rh_field_dict
)

In [91]:
equal(sqlq, ormq)

Equal ✔️


In [92]:
orm_to_df(ormq[:3])

,order_number,order_date,required_date,shipped_date,status,comments,customer_number,customer_name,contact_last_name,contact_first_name,phone,address_line1,city,state,postal_code,country
0,10100,2003-01-06,2003-01-13,2003-01-10,Shipped,None,363,Online Diecast Creations Co.,Young,Dorothy,6035558647,2304 Long Airport Avenue,Nashua,NH,62005,USA
1,10101,2003-01-09,2003-01-18,2003-01-11,Shipped,Check on availability.,128,"Blauer See Auto, Co.",Keitel,Roland,+49 69 66 90 2555,Lyonerstr. 34,Frankfurt,None,60528,Germany
2,10102,2003-01-10,2003-01-18,2003-01-14,Shipped,None,181,Vitachrome Inc.,Frick,Michael,2125551500,2678 Kingston Rd.,NYC,NY,10022,USA


### Exercise 2
Choose the correct JOIN clause to select all records from the two tables where there is a match in both tables.

In [93]:
q = """
Select * FROM orders o
INNER JOIN customers c
on o.customerNumber == c.customerNumber
"""
sqlq = sql_raw(q)
ji = sql_to_df(sqlq)
print(ji.shape)
ji.head(2)

(326, 16)


,orderNumber,orderDate,requiredDate,shippedDate,status,comments,customerNumber,customerName,contactLastName,contactFirstName,phone,addressLine1,city,state,postalCode,country
0,10100,2003-01-06,2003-01-13,2003-01-10,Shipped,None,363,Online Diecast Creations Co.,Young,Dorothy,6035558647,2304 Long Airport Avenue,Nashua,NH,62005,USA
1,10101,2003-01-09,2003-01-18,2003-01-11,Shipped,Check on availability.,128,"Blauer See Auto, Co.",Keitel,Roland,+49 69 66 90 2555,Lyonerstr. 34,Frankfurt,None,60528,Germany


In [94]:
lh_fields, rh_field_dict = get_all_join_fields(Orders, Customers, "customer_number")
ormq = (
    Orders.objects.filter(customer_number__isnull=False)
    .select_related("customer_number")
    .values(*lh_fields, **rh_field_dict)
)

In [95]:
equal(sqlq, ormq)

Equal ✔️


In [96]:
orm_to_df(ormq[:3])

,order_number,order_date,required_date,shipped_date,status,comments,customer_number,customer_name,contact_last_name,contact_first_name,phone,address_line1,city,state,postal_code,country
0,10100,2003-01-06,2003-01-13,2003-01-10,Shipped,None,363,Online Diecast Creations Co.,Young,Dorothy,6035558647,2304 Long Airport Avenue,Nashua,NH,62005,USA
1,10101,2003-01-09,2003-01-18,2003-01-11,Shipped,Check on availability.,128,"Blauer See Auto, Co.",Keitel,Roland,+49 69 66 90 2555,Lyonerstr. 34,Frankfurt,None,60528,Germany
2,10102,2003-01-10,2003-01-18,2003-01-14,Shipped,None,181,Vitachrome Inc.,Frick,Michael,2125551500,2678 Kingston Rd.,NYC,NY,10022,USA


### Exercise 3
Choose the correct JOIN clause to select all the records from the Customers table plus all the matches in the Orders table.

In [97]:
q = """
Select * FROM orders o
RIGHT OUTER JOIN customers c
on c.customerNumber == o.customerNumber
"""
sqlq = sql_raw(q)
ji = sql_to_df(sqlq)
print(ji.shape)
ji.head(2)

(350, 16)


,orderNumber,orderDate,requiredDate,shippedDate,status,comments,customerNumber,customerName,contactLastName,contactFirstName,phone,addressLine1,city,state,postalCode,country
0,10100.0,2003-01-06,2003-01-13,2003-01-10,Shipped,None,363,Online Diecast Creations Co.,Young,Dorothy,6035558647,2304 Long Airport Avenue,Nashua,NH,62005,USA
1,10101.0,2003-01-09,2003-01-18,2003-01-11,Shipped,Check on availability.,128,"Blauer See Auto, Co.",Keitel,Roland,+49 69 66 90 2555,Lyonerstr. 34,Frankfurt,None,60528,Germany


In [98]:
# Couldnt find any proper right join in djorm
lh_fields, rh_field_dict = get_all_join_fields(Customers, Orders, "orders")
ormq = (
    Customers.objects.select_related("orders")
    .order_by(F("orders__order_number").asc(nulls_last=True))
    .values(*lh_fields, **rh_field_dict)
)
ormdf = orm_to_df(ormq)

In [99]:
equal(sqlq, ormq)

Equal ✔️


In [100]:
orm_to_df(ormq[:3])

,customer_number,customer_name,contact_last_name,contact_first_name,phone,address_line1,city,state,postal_code,country,order_number,order_date,required_date,shipped_date,status,comments
0,363,Online Diecast Creations Co.,Young,Dorothy,6035558647,2304 Long Airport Avenue,Nashua,NH,62005,USA,10100,2003-01-06,2003-01-13,2003-01-10,Shipped,None
1,128,"Blauer See Auto, Co.",Keitel,Roland,+49 69 66 90 2555,Lyonerstr. 34,Frankfurt,None,60528,Germany,10101,2003-01-09,2003-01-18,2003-01-11,Shipped,Check on availability.
2,181,Vitachrome Inc.,Frick,Michael,2125551500,2678 Kingston Rd.,NYC,NY,10022,USA,10102,2003-01-10,2003-01-18,2003-01-14,Shipped,None


# GROUP BY
### Exercise 1
List the number of customers in each country.

In [101]:
sqlq = sql_raw("SELECT count(customerNumber) as cnt FROM  customers GROUP BY country;")
ormq = Customers.objects.values("country").annotate(cnt=Count("country")).values("cnt")
equal(sqlq, ormq)

Equal ✔️


In [102]:
orm_to_df(ormq[:3])

,cnt
0,5
1,2
2,2


### Exercise 2
List the number of customers in each country, ordered by the country with the most customers first.

In [105]:
sqlq = sql_raw(
    "SELECT country, count(customerNumber) as cnt FROM customers GROUP BY country ORDER BY count(customerNumber) desc;"
)
ormq = (
    Customers.objects.values("country")
    .annotate(cnt=Count("country"))
    .order_by("-cnt")
    .values("country", "cnt")
)
equal(sqlq, ormq)

Equal ✔️


In [106]:
orm_to_df(ormq[:3])

,country,cnt
0,USA,36
1,Germany,13
2,France,12
